In [0]:
%sql
USE CATALOG `adbliga1futbol`;


In [0]:
spark.sql("""
CREATE STORAGE CREDENTIAL liga1_credential
WITH AZURE_MANAGED_IDENTITY
COMMENT 'Credencial para Liga1 Perú'
""")




In [0]:
# Databricks notebook source
# ==========================================================
# CREACIÓN UNITY CATALOG: liga1_catalog
# Proyecto: Liga 1 Perú
# Autor: Oscar García Del Águila
# ==========================================================

import env_setup
from pyspark.sql import SparkSession
from utils_liga1 import setup_adls, get_dbutils, get_abfss_path

In [0]:
# ----------------------------------------------------------
# CONFIGURACIÓN ADLS
# ----------------------------------------------------------
setup_adls()
spark = SparkSession.builder.getOrCreate()
dbutils = get_dbutils()

In [0]:
# ----------------------------------------------------------
# RUTA BASE PARA EL CATÁLOGO
# ----------------------------------------------------------
ruta_catalog = get_abfss_path("udv/")
print(f"Ruta base para el catálogo: {ruta_catalog}")

In [0]:
# ----------------------------------------------------------
# CREAR CARPETA EN ADLS SI NO EXISTE
# ----------------------------------------------------------
try:
    # Intentar listar el directorio para validar existencia
    dbutils.fs.ls(ruta_catalog)
    print("Carpeta 'udv/' ya existe en ADLS.")
except Exception:
    print("Carpeta 'udv/' no existe, creando...")
    dbutils.fs.mkdirs(ruta_catalog)
    print("Carpeta 'udv/' creada correctamente en ADLS.")

In [0]:
# ----------------------------------------------------------
# CREAR CATÁLOGO SI NO EXISTE
# ----------------------------------------------------------
query_catalog = f"""
CREATE CATALOG IF NOT EXISTS liga1_catalog
MANAGED LOCATION '{ruta_catalog}';
"""
spark.sql(query_catalog)
print("Catálogo 'liga1_catalog' creado o verificado correctamente.")

In [0]:
# ----------------------------------------------------------
# CREAR SCHEMA 'udv' DENTRO DEL CATÁLOGO
# ----------------------------------------------------------
query_schema = """
CREATE SCHEMA IF NOT EXISTS liga1_catalog.udv
COMMENT 'Capa UDV de la Liga 1 Perú (entidades limpias y homologadas).';
"""
spark.sql(query_schema)
print("Esquema 'udv' creado correctamente dentro del catálogo.")

In [0]:
# ----------------------------------------------------------
# VALIDACIÓN FINAL
# ----------------------------------------------------------
print("📋 Catálogos disponibles:")
spark.sql("SHOW CATALOGS").show()

print("📋 Esquemas en liga1_catalog:")
spark.sql("SHOW SCHEMAS IN liga1_catalog").show()